Before we begin, let us execute the below cell to display information about the NVIDIA® CUDA® driver and the GPUs running on the server by running the `nvidia-smi` command. To do this, execute the cell block below by clicking on it with your mouse, and pressing Ctrl+Enter, or pressing the play button in the toolbar above. You should see some output returned below the grey cell.

In [ ]:
!nvidia-smi

## Exercise 4

### Learning objectives

The **goal** of this lab is to:

- Learn how to identify redundant memory copies via NVIDIA Nsight™ Systems
- Learn how to improve efficiency by reducing extra data copies via OpenACC data directives
- Learn how to use NVIDIA HPC compiler feedback for guidance on where to insert OpenACC data directives
- Apply data directives to the parallel application, benchmark and profile the application

We do not intend to cover:

- The OpenACC programming model
- Advanced optimization techniques in detail

Let's inspect the profiler report from the previous exercise. From the "Timeline view" on the top pane, double-click on "CUDA" from the function table on the left and expand it. Zoom in on the timeline and you can see a pattern similar to the screenshot below. The blue boxes are the compute kernels and each of these groupings of kernels is surrounded by purple and green boxes (annotated with a green rectangle) representing data movements. If you hover your mouse over each box, you can see more details.

What this graph is showing is that there is a lot of data movement between GPU and CPU.
    
<img src="images/nsys_data_mv.png">

The compiler feedback we collected earlier tells us quite a bit about data movement too. If we look again at the compiler feedback from above, we see the following (the lines in the compiler feedback might be slightly different for you).

<img src="images/cfeedback3-1_.png" width="80%" height="80%">

The compiler feedback is telling us that the compiler has inserted data movement around our parallel region at line 278 which copies the `hy_dens_cell`, `hy_dens_theta_cell`, and `state` arrays in and out of GPU memory and also copies `flux` array out. 

The compiler can only work with the information we provide. It knows we need the `hy_dens_cell`, `hy_dens_theta_cell`, `state`, and `flux` arrays on the GPU for the accelerated section within the  `compute_tendencies_x` function, but we didn't tell the compiler anything about what happens to the data outside of those sections. Without this knowledge, the compiler has to copy the full arrays to the GPU and back to the CPU for each accelerated section. This is a good deal of unnecessary data transfers. 

Ideally, we would want to move the data (example: `hy_dens_cell`, `hy_dens_theta_cell`, `state` arrays) to the GPU at the beginning, and only transfer it back to the CPU at the end (if needed). And as for the `flux` array in this example, we do not need to copy any data back and forth. So we only need to create space on the device (GPU) for this array. 

We need to give the compiler information about how to reduce the extra and unnecessary data movement. By adding an OpenACC `data` directive to a structured code block, the compiler will know how to manage data according to the clauses. For information on the data directive clauses, please visit [OpenACC 3.0 Specification](https://www.openacc.org/sites/default/files/inline-images/Specification/OpenACC.3.0.pdf).

We added `data` directives to the code. You can inspect the code at <b>[miniWeather_openacc.cpp](../source_code/lab4/miniWeather_openacc.cpp)</b>. Once done, let's compile it and review the compiler feedback and profile again.

In [ ]:
!cd ../source_code/lab4 && make clean && make

Start inspecting the compiler feedback and see if it applied the optimizations (the lines in the compiler feedback might be slightly different for you). Here is the screenshot of expected compiler feedback after adding the `data` directives. You can see that on line 281, the compiler is generating default present for `hy_dens_cell`, `hy_dens_theta_cell`, `state`, and `flux` arrays. In other words, it is assuming that data is present on the GPU and it only copies data to the GPU only if the data do not exist.

<img src="images/cfeedback4_.png" width="90%" >

Now, **Profile** the code with Nsight Systems command line `nsys`.

In [ ]:
!cd ../source_code/lab4 && nsys profile -t nvtx,openacc --stats=true --force-overwrite true -o miniWeather_4 ./miniWeather

Download and save the report file by holding down <mark>Shift</mark> and <mark>right-clicking</mark> [Here](../source_code/lab4/miniWeather_4.nsys-rep), then choosing <mark>Save Link As</mark>. Once done open the report via the Nsight System user interface (UI)  locally. Now, have look at the expected output example below:

<img src="images/nsys_fast_mv_.png">

Look at the data movements annotated with green and purple colors and compare them with the previous versions. We have accelerated the application and reduced the execution time by eliminating the unnecessary data transfers between CPU and GPU.

Let's look at the NVTX ranges to see how much speedup we achieved after multiple optimizations.

|   | Serial | Parallel (lab2) | Parallel (lab3) | Parallel (lab4) |
| --- | ----------- |----------- |----------- |----------- |
| Total | 27.66 s |157.47 s | 7.14 s |1.27 s |
| While | 27.66 s  |157.09 s | 6.77 s |926.69 ms |
| perform_timestep| 22.72 s |131.09 ms |  4.89 ms |0.78399 ms | 

**Note**: The next exercise gives an overview of the introduction to NVIDIA Nsight Compute tool and is optional.

## Post-Lab Summary

If you would like to download this lab for later viewing, it is recommended you go to your browser's file menu (not the Jupyter notebook file menu) and save the complete web page.  This will ensure the images are copied down as well. You can also execute the following cell block to create a zip file of the files you have been working on, and download it with the link below.

In [ ]:
%%bash
cd ..
rm -f _profiler_files.zip
zip -r _profiler_files.zip *

**After** executing the above zip command, you should be able to download and save the zip file by holding down <mark>Shift</mark> and <mark>right-clicking</mark> [Here](../_profiler_files.zip), then choosing <mark>Save Link As</mark>. 

-----

# <p style="text-align:center; border:3px; border-style:solid; border-color:#FF0000  ; padding: 1em"> <a href=../_start_profiling.ipynb>HOME</a>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style="float:center"> <a href=profiling_lab5.ipynb>NEXT</a></span> </p>

-----

# Links and Resources

[OpenACC API Guide](https://www.openacc.org/sites/default/files/inline-files/OpenACC%20API%202.6%20Reference%20Guide.pdf)

[NVIDIA Nsight System](https://docs.nvidia.com/nsight-systems/)

[CUDA Toolkit Download](https://developer.nvidia.com/cuda-downloads)

**NOTE**: To be able to see the Nsight System profiler output, please download the latest version of Nsight Systems from [here](https://developer.nvidia.com/nsight-systems).


Don't forget to check out additional [Open Hackathons Resources](https://www.openhackathons.org/s/technical-resources) and join our [OpenACC and Hackathons Slack Channel](https://www.openacc.org/community#slack) to share your experience and get more help from the community.


--- 

## Licensing 

Copyright © 2022 OpenACC-Standard.org.  This material is released by OpenACC-Standard.org, in collaboration with NVIDIA Corporation, under the Creative Commons Attribution 4.0 International (CC BY 4.0). These materials may include references to hardware and software developed by other entities; all applicable licensing and copyrights apply.